# Computing simplicial homology
The code below is based on work Chandradeep Chowdhury did for his senior project

In [87]:
import sympy as sp
#1. Calculate the k-chains
#2. Calculate the boundary maps 
#3. Calculate the requested homology groups
#4. Calculate the rank of the homology group. 

In [88]:
#Helper functions:
#Prints a newline
def newline():
    print("\n");

In [89]:
#Find weight at index
def weight(index):
    return (-1)**index;

#Function (Forward direction of isomorphism between linear maps and matrices) to convert the boundary homomorphism to a matrix in the natural bases of k-chains:
def morphism(C_k, C_kplus1):
    ret = [];
    for x in C_kplus1:
        out = [];
        for i in range(0, len(x)):
            temp = x.copy();
            del temp[i];
            out.append(temp);
        for y in C_k:
            if y in out:
                ret.append(weight(out.index(y)));
            else:
                ret.append(0);
    return ret;

In [90]:
#load the complexes into a list: (below are some examples)
#The script does not currently check if the list is a valid simlicial complex. 
sc1 = [[],[1],[2],[3],[4],
[1,2],[1,3],[2,3],[2,4],[3,4],
[2,3,4]];

sc2 = [[],[1],[2],[3],
[1,2],[1,3],[2,3]
];

sc3 = [[], [1], [2], [3], [1,2]];

sc4 = [[], ["a"],["b"],["c"],["d"],["e"],["f"],["b","c"],["c","d"],["c","e"],["d","e"],["c","d","e"],["d","f"],["e","f"]]; 


In [91]:
def shom(sc): 
    print("\nSimplicial Complex: \n", sc, "\n");

    #initialize the hashmap
    C = {};

    #load the complex into a hashmap to seperate the k-chains:
    #While in theory we do not need C_-1, it was helpful for the algorithm.
    for simplex in sc:
        if (len(simplex)-1) not in C.keys():
            C[len(simplex)-1] = [simplex];
        else:
            C[len(simplex)-1].append(simplex); 
    
    #Little trick to add an empty k-chain at the end. 
    for i in C:
        continue;
    C[i+1] = [[]];

      
    print("Simplicial k-chains:");
    for i in C:
        print("C_", i, " =  span {", ', '.join(map(str,C[i])), "}", sep = '');


    #Calculate the boundary homomorphisms 
    d = [];

    for i in C:
        if i == 0:
            d.append(sp.zeros(1, len(C[i])));
        elif i > 0:
            M = sp.Matrix(len(C[i]), len(C[i-1]), morphism(C[i-1], C[i])); #Converting the map to matrix form here. 
            d.append(M.T);
    
    newline();
    
    #calculate homology group and its rank:

    #This algorithm may not produce the same quotient basis for you would find manually

    print("kth Homology:")
    for i in range(0, len(d) - 1):

        ker = d[i].nullspace();
        img = d[i+1].columnspace();

        y = sp.Matrix();
        
        for z in img:
            y = y.row_join(z);

        for x in ker:
            y = y.row_join(x);

        pivots = (y.rref())[1];

        span = [];

        for k in range(len(img), y.shape[1]):
            if k in pivots:
                span.append(y.col(k));
        
        
        #backward direction of the isomorphism 
        veclist = []
        for vector in span:
            rep = ""
            for j in range(len(vector)):
                    if len(rep) == 0:
                        if vector[j] == 0:
                            continue
                        elif vector[j] == 1:
                            rep += str(C[i][j])
                        elif vector[j] == -1:
                            rep += " - " + str(C[i][j])
                    else:
                        if vector[j] == 0:
                            continue
                        elif vector[j] == 1:
                            rep += " + " + str(C[i][j])
                        elif vector[j] == -1:
                            rep += " - " + str(C[i][j])
            veclist.append(rep)
        
        
        
        print("H_" + str(i) + " = span{" + ', '.join(veclist) + "}")

        print("Rank of H_", i, " = ", len(ker) - len(img), sep ='');
        newline();
    
    return;

In [92]:
#Call the function here: 
shom(sc4);


Simplicial Complex: 
 [[], ['a'], ['b'], ['c'], ['d'], ['e'], ['f'], ['b', 'c'], ['c', 'd'], ['c', 'e'], ['d', 'e'], ['c', 'd', 'e'], ['d', 'f'], ['e', 'f']] 

Simplicial k-chains:
C_-1 =  span {[]}
C_0 =  span {['a'], ['b'], ['c'], ['d'], ['e'], ['f']}
C_1 =  span {['b', 'c'], ['c', 'd'], ['c', 'e'], ['d', 'e'], ['d', 'f'], ['e', 'f']}
C_2 =  span {['c', 'd', 'e']}
C_3 =  span {[]}


kth Homology:
H_0 = span{['a'], ['b']}
Rank of H_0 = 2


H_1 = span{ - ['c', 'd'] + ['c', 'e'] - ['d', 'f'] + ['e', 'f']}
Rank of H_1 = 1


H_2 = span{}
Rank of H_2 = 0


